In [1]:
import sys
print(sys.path)

['C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\python36.zip', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\DLLs', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\lib', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp', '', 'C:\\Users\\ssotn\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\lib\\site-packages', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\ssotn\\.ipython']


In [2]:
import spacy
#!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_md

nlp_en = spacy.load('en_core_web_md', disable=['ner', 'textcat'])

# Колокації

```
продовжте синонімний ряд дієслів: "say", "tell", "speak", "claim", "communicate"
```

Take the hi- and medium-orange synonims from https://www.thesaurus.com/browse/say:

add, announce, answer, assert, claim, convey, declare, deliver, disclose, do, estimate, express, maintain, mention, read, repeat, reply, report, respond, reveal, speak, state, suggest, tell, voice, affirm, allege, communicate, conjecture, divulge, flap, gab, guess, imagine, imply, jaw, judge, lip, opine, orate, perform, pronounce, rap, recite, rehearse, relate, remark, render, rumor, spiel, utter, verbalize, yak

In [3]:
syn_words = ["say", "add", "announce", "answer", "assert", "claim", "convey", "declare", "deliver",
        "disclose", "do", "estimate", "express", "maintain", "mention", "read", "repeat", "reply",
        "report", "respond", "reveal", "speak", "state", "suggest", "tell", "voice", "affirm", 
        "allege", "communicate", "conjecture", "divulge", "flap", "gab", "guess", "imagine", 
        "imply", "jaw", "judge", "lip", "opine", "orate", "perform", "pronounce", "rap", "recite", 
        "rehearse", "relate", "remark", "render", "rumor", "spiel", "utter", "verbalize", "yak"]

In [4]:
from collections import namedtuple

LyStatEntry = namedtuple('LyStatEntry', ['lemma', 'ly_adv'])
    
def make_ly_stats(text: str):
    res = []
    doc = nlp_en(text)
    for t in doc:
        if t.pos_!='ADV':
            continue
        if not str(t).lower().endswith('ly'):
            continue
        #print(t)
        parent = None
        head = t.head
        if head==None or head==t:
            continue
        if t.dep_=='conj':
            head = head.head
        #print(head)
        #print(f"{head}:{head.dep_} -> {t}:{t.dep_}")
        if head.pos_=='VERB' and head.lemma_ in syn_words:
            #print(">>"+head)
            res.append(LyStatEntry(lemma = head.lemma_, ly_adv= str(t).lower()))
        # elif head.
            
    return res

In [5]:
print(make_ly_stats("""-- Sheryl Crow at the Huffington Post9 ) " I hear about Tony Snow and say to myself , well , stand up every day , lie to the American people at the behest of your dictator-esque boss and well , how could a cancer NOT grow in you ."""))
print(make_ly_stats("""Work for Fox News , spinning the truth in to a billion knots and how can your gut not rot ?"""))
print(make_ly_stats("speak somebody dearly"))
print(make_ly_stats("honor somebody highly")) 
print(make_ly_stats("claim somebody greatly"))
print(make_ly_stats("""Speaking a week after Bhutto 's assassination in a shooting and suicide bombing , Musharraf denied there had been a security lapse and implied that Bhutto , who was greeting supporters through the sunroof of her armored vehicle at the time of the attack , was partly responsible ."""))
print(make_ly_stats("Mariana replied me sincerely and hopefully."))

[]
[]
[LyStatEntry(lemma='speak', ly_adv='dearly')]
[]
[LyStatEntry(lemma='claim', ly_adv='greatly')]
[]
[LyStatEntry(lemma='reply', ly_adv='sincerely'), LyStatEntry(lemma='reply', ly_adv='hopefully')]


In [6]:
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

def process_blog_texts():
    with open('blog2008.txt', 'r', encoding="utf-8") as inp:
        total = 0
        stat = defaultdict(lambda: defaultdict(int))
        for head in tqdm(inp, total=303995):
            # if total>5000:
            #     break
            head = head.strip()
            stat_items = make_ly_stats(head)
            total += 1
            for item in stat_items:
                stat[item.lemma][item.ly_adv] += 1
        print(f"Total lines: {total}, 'say'-verbs found: {len(stat)}")
        return stat

In [7]:
# %load_ext line_profiler
# %lprun -f make_ly_stats process_blog_texts()
stat = process_blog_texts()
# print(stat)


Total lines: 303994, 'say'-verbs found: 45


In [8]:
def write_statistics(stat):
    with open('collocations.txt', 'w', encoding="utf-8") as outp:
        for k, v in stat.items():
            s_values = sorted(v.items(), key=lambda x: x[1], reverse=True)
            values_str = str(s_values[:10])[1:-1]
            print(f"{k}: {values_str}", file=outp)

In [9]:
write_statistics(stat)